In [1]:
import os, csv, random, re, nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
word2vec = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
print("Loaded word vectors successfully!")

Loaded word vectors successfully!


In [3]:
stop_words = set()
with open('stops.txt', 'r') as f:
    for line in f:
        stop_words.add(line.strip())
for w in stopwords.words('english'):
    stop_words.add(w)
total_train = 10000
total_test = 2000

train_idx = range(total_train)
test_idx = range(total_test)

In [4]:
def prefixReplace(match):
    prefix, stem = match.group(1), match.group(2)
    temp = prefix + stem
    if not stem in stop_words:
        temp += ' ' + stem
    return temp

def hyphenReplace(match):
    temp = match.group()
    li = temp.split('-')
    temp = temp.replace('-', '')
    for item in li:
        if not item in stop_words:
            temp += ' ' + item
    return temp

In [5]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    wnl = WordNetLemmatizer()
    p_prefix = re.compile(r'\b(a|an|ante|anti|auto|circum|co|com|con|contra|contro|de|dis|en|em|ex|extra|fore|hetero|homo|homeo|hyper|il|im|in|ir|inter|intra|intro|macro|micro|mid|mis|mono|non|omni|over|post|pre|pro|re|semi|sub|super|sym|syn|trans|tri|un|under|uni)-([a-z])+\b', re.I)
    p_hyphen = re.compile(r'\b(\w+-)+\w+\b')
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        li = []
        with open(path) as f:
            for line in f:
                line = line.strip()
                line = line.lower()
                # expand stem with hyphen prefix
                line = p_prefix.sub(prefixReplace, line)
                # expand hyphenated word
                line = p_hyphen.sub(hyphenReplace, line)
                line = line.replace(':', ' ')
                line = line.replace('\'s', '')
                line = line.replace(',', ' ')
                line = line.replace('.', ' ')
                li += word_tokenize(line)
                li = [wnl.lemmatize(x) for x in li]
        docs.append(li)
    return docs

def doc_to_vec(li, word2vec):
    # get list of word vectors in sentence
    word_vecs = []
    for w in li:
        if w not in word2vec.vocab:
#             print('not found:',w)
            pass
        elif w in stop_words:
#             print('stop word:', w)
            pass
        else:
            word_vecs.append(word2vec.get_vector(w))
    if not li:
        return np.zeros(300)
    # return average
    return np.stack(word_vecs).mean(0)

In [6]:
# build description matrices
d_train_dev = parse_descriptions("../data/descriptions_train", num_doc=total_train)
test_desc = parse_descriptions("../data/descriptions_test", num_doc=total_test)
d_train = np.array([doc_to_vec(d_train_dev[i], word2vec) for i in train_idx])
d_test = np.array([doc_to_vec(test_desc[i], word2vec) for i in test_idx])
d_train_dev.clear()
test_desc.clear()
del d_train_dev, test_desc
np.save('word2vec_train_description.npy', d_train)
np.save('word2vec_test_description.npy', d_test)
print("Built all description matrices!")
print("d_train shape:", d_train.shape)
print("d_test shape:", d_test.shape)

Built all description matrices!
d_train shape: (10000, 300)
d_test shape: (2000, 300)


In [7]:
# build test matrices
t_train_dev = parse_descriptions("../data/tags_train", num_doc=total_train)
test_tag = parse_descriptions("../data/tags_test", num_doc=total_test)
t_train = np.array([doc_to_vec(t_train_dev[i], word2vec) for i in train_idx])
t_test = np.array([doc_to_vec(test_tag[i], word2vec) for i in test_idx])

t_train_dev.clear()
test_tag.clear()
del t_train_dev, test_tag
np.save('word2vec_train_tag.npy', t_train)
np.save('word2vec_test_tag.npy', t_test)
print("Built all tag matrices!")
print("t_train shape:", t_train.shape)
print("t_test shape:", t_test.shape)

Built all tag matrices!
t_train shape: (10000, 300)
t_test shape: (2000, 300)


In [8]:
def parse_features(features_path):
    vec_map = {}
    with open(features_path) as f:
        for row in csv.reader(f):
            img_id = int(row[0].split("/")[1].split(".")[0])
            vec_map[img_id] = np.array([float(x) for x in row[1:]])
    return np.array([v for k, v in sorted(vec_map.items())])

# build feature matrices
# p = np.random.randn(1000, 200)
f_train_dev = parse_features("../data/features_train/features_resnet1000_train.csv")# @ p
f_train = f_train_dev[train_idx]
f_test = parse_features("../data/features_test/features_resnet1000_test.csv")# @ p
f_test = f_test[test_idx]

del f_train_dev
np.save('word2vec_train_1000feature.npy', f_train)
np.save('word2vec_test_1000feature.npy', f_test)
print("Built all feature vec matrices!")
print("f_train shape:", f_train.shape)
print("f_test shape:", f_test.shape)

Built all feature vec matrices!
f_train shape: (10000, 1000)
f_test shape: (2000, 1000)


In [9]:
fi_train_dev = parse_features("../data/features_train/features_resnet1000intermediate_train.csv")
fi_train = fi_train_dev[train_idx]
fi_test = parse_features("../data/features_test/features_resnet1000intermediate_test.csv")# @ p
fi_test = fi_test[test_idx]

del fi_train_dev
np.save('word2vec_train_2048feature.npy', fi_train)
np.save('word2vec_test_2048feature.npy', fi_test)
print("Built all intermediate feature vec matrices!")
print("fi_train shape:", fi_train.shape)
print("fi_test shape:", fi_test.shape)

Built all intermediate feature vec matrices!
fi_train shape: (10000, 2048)
fi_test shape: (2000, 2048)
